In [24]:
from pyspark import SparkConf
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import isnull, avg, min, date_format
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType, StructField
import pyspark.sql.functions as F
import pyspark.pandas as ps
ps.set_option('compute.ops_on_diff_frames', True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
POSTS_FP = "s3://de3-extract-results/naver_cafe_posts.csv"
VIEW_TABLE_DIR = "s3://softter-de3-quicksight/"

posts_df = spark.read.option("header", True).csv(POSTS_FP)
# comments_df = spark.read.option("header", True).csv(COMMENTS_FP)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
posts_df.show(1, vertical=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0--------------------------------------
 id         | 1129062                          
 title      | 전기차 화재                      
 content    | [뉴스토리] 전기차 화재의 비밀... 
 likes      | 1.0                              
 url        | https://cafe.nave...             
 author     | 둥이아부지I봉고I광주             
 views      | 790                              
 created_at | 2023-01-28 17:18:00              
 updated_at | null                             
only showing top 1 row

In [27]:
posts_df = posts_df.withColumn("created_day", F.to_date(posts_df["created_at"], "yyyy-MM-dd HH:mm:ss"))
type(posts_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.sql.dataframe.DataFrame'>

In [28]:
posts_df.show(1, vertical=True)
posts_df = posts_df.fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0---------------------------------------
 id          | 1129062                          
 title       | 전기차 화재                      
 content     | [뉴스토리] 전기차 화재의 비밀... 
 likes       | 1.0                              
 url         | https://cafe.nave...             
 author      | 둥이아부지I봉고I광주             
 views       | 790                              
 created_at  | 2023-01-28 17:18:00              
 updated_at  | null                             
 created_day | 2023-01-28                       
only showing top 1 row

In [29]:
posts_group_by_day_df = posts_df.groupby("created_day").agg({'title': 'count', 'likes': 'sum', 'views': 'sum'})
new_col_names = ['created_day', 'count_title', 'sum_views', 'sum_likes']
posts_group_by_day_df = posts_group_by_day_df.toDF(*new_col_names)
posts_group_by_day_df = posts_group_by_day_df.fillna(0)
posts_group_by_day_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-----------+---------+---------+
|created_day|count_title|sum_views|sum_likes|
+-----------+-----------+---------+---------+
| 2022-08-02|          1|      0.0|      1.0|
| 2023-09-14|          1|    342.0|      0.0|
| 2024-06-12|          1|      0.0|      1.0|
| 2021-09-14|          1|      0.0|      0.0|
| 2021-04-24|          2|    541.0|      0.0|
| 2021-07-23|          2|    146.0|      1.0|
| 2022-07-07|          1|    418.0|      0.0|
| 2023-01-01|          2|    979.0|      0.0|
| 2022-12-20|          2|   1164.0|      0.0|
| 2024-05-19|          1|      0.0|      1.0|
| 2021-02-02|          6|   1723.0|      8.0|
| 2024-03-28|          2|      0.0|      5.0|
| 2022-01-09|          1|    260.0|      0.0|
| 2022-04-27|          1|    139.0|      0.0|
| 2022-07-05|          1|    726.0|      4.0|
| 2023-07-04|          1|      0.0|      0.0|
| 2022-01-18|          1|    616.0|      0.0|
| 2021-12-14|          2|      0.0|      0.0|
| 2023-08-03|          1|    338.0

In [30]:
posts_group_by_day_df = posts_group_by_day_df.to_pandas_on_spark()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:5252: FutureWarning: DataFrame.to_pandas_on_spark is deprecated. Use DataFrame.pandas_api instead.
  FutureWarning,

In [31]:
# posts_group_by_day_df.count_title.ewm(alpha=0.6).mean()
type(posts_group_by_day_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.pandas.frame.DataFrame'>

In [32]:
posts_group_by_day_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  created_day  count_title  sum_views  sum_likes
0  2022-08-02            1        0.0        1.0
1  2023-09-14            1      342.0        0.0
2  2024-06-12            1        0.0        1.0
3  2021-09-14            1        0.0        0.0
4  2021-04-24            2      541.0        0.0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
posts_group_by_day_df = posts_group_by_day_df.sort_values('created_day')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
posts_group_by_day_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    created_day  count_title  sum_views  sum_likes  count_smoothed
390  2020-01-09            1      553.0        0.0        1.652083
83   2020-01-12            1      850.0        4.0        1.005409
227  2020-01-19            1      728.0        1.0        1.022999
691  2020-02-07            1        0.0        1.0        4.157724
311  2020-03-21            2      857.0        0.0        1.739562

In [37]:
# start date is min date
date_min=posts_group_by_day_df.agg({'created_day': 'min'})[0]
date_max=posts_group_by_day_df.agg({'created_day': 'max'})[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
full_date_df = ps.date_range(start=date_min, end=date_max, freq='D', name='date').to_frame()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
output_df = ps.merge(full_date_df, posts_group_by_day_df, left_on = 'date', right_on='created_day', how='left').sort_values(by=['date'])

count_title_df = posts_group_by_day_df[['count_title']]
posts_group_by_day_df['count_smoothed'] = count_title_df.ewm(alpha=0.7, min_periods=1).mean()


output_df = output_df.drop(columns=['created_day'])
output_df = output_df.fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
output_df.to_csv(VIEW_TABLE_DIR+"dashboard_example", mode='overwrite', header=True)
posts_df.coalesce(1).write.option("header", True).mode('overwrite').csv(VIEW_TABLE_DIR+"naver_cafe_posts")    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)